## Modify to make the script for putting "Your" model into the BoxSDK API

In [1]:
# Import two classes from the boxsdk module - Client and OAuth2
from boxsdk import Client, OAuth2
from boxsdk.network.default_network import DefaultNetwork
from boto.dynamodb.condition import NULL
from BOXFile import BOXFile as bf
import pandas as pd
import io
import os
import numpy as np
from io import StringIO
import h5py
from progressbar import ProgressBar
import pickle


os.chdir('C:/Users/rjohnson18/Box/CIROH/Evaluation/LULC_Streamflow/')
cwd = os.getcwd()

## Client BoxSDK SPI access

In [2]:
# Define client ID, client secret, and developer token.
CLIENT_ID = None
CLIENT_SECRET = None
ACCESS_TOKEN = None

# Read app info from text file
with open('./Streamflow_App/app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    ACCESS_TOKEN = app_cfg.readline()

# Create OAuth2 object. It's already authenticated, thanks to the developer token.
oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

# Create the authenticated client
client = Client(oauth2)

In [3]:
# Get information about the logged in user (that's whoever owns the developer token)
my = client.user(user_id = 'me').get()
print(my.name)
print(my.login)
print(my.avatar_url)

Ryan Johnson
rjohnson18@ua.edu
https://alabama.app.box.com/api/avatar/large/16500719171


## Upload data into App via BoxSDK

We will want to create a dictionary of folder names, folder ids, file names, file ids to speed up the search querying.

Make a function for loading data from Box Drive, creating a subfolder, saving subfolder id/name as dictionary, and saving data to BoxSDK

In [100]:
def Drive_to_SDK(Folder_Dictionary, Reach_Dictionary, data):
    
    #working out of the folder_id = '0' root
    root_folder = client.folder(folder_id='0')

    #get list of state .h5 files
    data_path = cwd+'/Data/'+data
    state_list = os.listdir(data_path)
    #remove states already completed
    #state_list.remove('NHD_segments_wi.h5')
   
    
    
    for state in state_list:
        #Make filepath to each state.h5
        filepath = data_path+'/'+state
        #Create a subfolder and name to store ___ datasets
        if state == 'NHD_segments_wi.h5':
            subfolder = pd.DataFrame()
            subfolder['id'] = '202149569763'
        else: 
            subfolder = root_folder.create_subfolder(state)
            print('Folder {0} created'.format(subfolder.get()['name']), 'with id: ', subfolder.id)
            Folder_Dictionary[state] = subfolder.id
        #Open each file and save to BoxSDK as csv
        with h5py.File(filepath, "r") as f:
            keys =  f.keys()
            pbar = ProgressBar()
            for key in pbar(keys):
                df = pd.read_hdf(filepath, key = key)
                s_buf = StringIO()
                df.to_csv(s_buf)
                
                if state == 'NHD_segments_wi.h5':
                    folder_id = '202149569763'
                else:
                    folder_id = subfolder.id # '201963057861'
                    
                
                try:
                    Searchable_filename = data+'_'+key+'.csv'
                    new_file = client.folder(folder_id).upload_stream(s_buf, Searchable_filename)
                    Reach_Dictionary[new_file.name[:-4]] = new_file.id
                    print(f'File "{new_file.name}" uploaded to Box with file ID {new_file.id}')
                except:
                    pass
                

In [ ]:
#Need to put dictionaries here and not run after initialization
#state_dictionary = {}
#NWM_dictionary = {}

In [101]:
#Make files in BoxSDK for all NWIS sites
Drive_to_SDK(state_dictionary, NWM_dictionary, 'NWM')

"POST https://upload.box.com/api/2.0/files/content" 409 475              |
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:22:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '301', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-manager-92gb', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'}
<No content or content unavailable for logging>

"POST https://upload.box.com/api/2.0/files/content" 409 475
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:22:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '338', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-m

"POST https://upload.box.com/api/2.0/files/content" 409 475
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:03 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '246', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-manager-j25t', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'}
<No content or content unavailable for logging>

"POST https://upload.box.com/api/2.0/files/content" 409 475              |
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:04 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '211', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-m

"POST https://upload.box.com/api/2.0/files/content" 409 475              |
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '222', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-manager-n6n0', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'}
<No content or content unavailable for logging>

"POST https://upload.box.com/api/2.0/files/content" 409 475
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:32 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '228', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-c-traffic-m

"POST https://upload.box.com/api/2.0/files/content" 409 475
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:57 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '329', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-manager-n6n0', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'}
<No content or content unavailable for logging>

"POST https://upload.box.com/api/2.0/files/content" 409 475              |
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:23:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '201', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-m

"POST https://upload.box.com/api/2.0/files/content" 409 475              |
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:24:24 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '207', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-b-traffic-manager-fxns', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"'}
<No content or content unavailable for logging>

"POST https://upload.box.com/api/2.0/files/content" 409 475
{'Server': 'nginx', 'Date': 'Thu, 06 Apr 2023 15:24:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '475', 'Cache-Control': 'no-cache, no-store', 'X-Envoy-Upstream-Service-Time': '222', 'Strict-Transport-Security': 'max-age=31536000', 'X-Box-Original-Ingress-ADC-Host': 'prod-c-traffic-m

...

File "NWM_5334435.csv" uploaded to Box with file ID 1184096741282


File "NWM_5334799.csv" uploaded to Box with file ID 1184096071486


File "NWM_5387929.csv" uploaded to Box with file ID 1184096619524


File "NWM_7887898.csv" uploaded to Box with file ID 1184096657843


File "NWM_7898927.csv" uploaded to Box with file ID 1184096273587


In [162]:

import pickle
f = open('./Streamflow_App/NWM_state_keys.pkl', 'wb')
pickle.dump(state_dictionary, f)
f.close()

n = open('./Streamflow_App/NWM_site_keys.pkl', 'wb')
pickle.dump(NWM_Dictionary, n)
n.close()


In [29]:
#Need to put the segment id dictionaries into a csv
dataname = 'NWIS_state'
dataid = dataname+'_id'
Searchable_filename = dataname+'_keys.csv'

#load dictionary
Dictionary = pickle.load(open(cwd+'/Streamflow_App/'+dataname+'_keys.pkl', 'rb'))

columns = [dataname, dataid]
DF = pd.DataFrame(columns = columns)

pbar = ProgressBar()
for site in pbar(Dictionary.keys()):
    #remove .csv from any id's
    if '.csv' in site:
        sitename = site[:-4]
    else:
        sitename = site
    siteinfo = [site, Dictionary[site]]
    #DF = DF.append(siteinfo, ignore_index = True)
    DF.loc[len(DF)] = siteinfo
    
#save as stringIO    
s_buf = StringIO()
DF.to_csv(s_buf)
new_file = client.folder('0').upload_stream(s_buf, Searchable_filename)
print(f'File "{new_file.name}" uploaded to Box with file ID {new_file.id}')

#updated_file = client.file('1184303261277').update_contents(s_buf)
#print(f'File "{updated_file.name}" with id: "{updated_file.id}" has been updated')
#File "NWM_site_keys.csv" uploaded to Box with file ID 1184296622892
#File "NWIS_site_keys.csv" uploaded to Box with file ID 1184303261277
#File "NWIS_site_keys_Conus.csv" uploaded to Box with file ID 1184330851812 -- use this one
#File 'streamstats' uploaded to Box with file ID '1182325029983'
#File "NWM_v2.1_state_keys.csv" uploaded to Box with file ID 1185045021944
#File "NWIS_state_keys.csv" uploaded to Box with file ID 1185051421057

100% |########################################################################|


File "NWIS_state_keys.csv" uploaded to Box with file ID 1185051421057


## Code for finding folder id from name

In [4]:
folders = [
    'WBD_01_HU2_GDB.gdb'
]

for folder_name in folders:
    #folder_name = 'NHD_segments_al.h5'
    results = client.search().query(query = folder_name,type = 'folder', limit = 1)

    for result in results:
        name = result.name
        if name == folder_name:
            print(f'The item ID is {result.id} and the item name is {name}')
            break
            
#The item ID is 202307017034 and the item name is NWM_testing_this_one

The item ID is 184102567655 and the item name is WBD_01_HU2_GDB.gdb


## Code for finding all files in a folder


In [ ]:
#load NWIS State Keys
#load dictionary
NWIS_State_Dict = pickle.load(open(cwd+'/Streamflow_App/NWIS_state_keys.pkl', 'rb'))
NWIS_State_Dict

In [87]:
r_dict = {}
pbar = ProgressBar()
#for key in pbar(state_dictionary.keys()):
#s = ['NWIS_sites_al.h5']
folder_id = 
for key in pbar(s):
    print(key)
    #streamstats_folder_id =  state_dictionary[key]#Folder_Dictionary['NWIS_sites_ar.h5'] # '201963057861'
    streamstats_folder_id =  NWIS_State_Dict[key]#Folder_Dictionary['NWIS_sites_ar.h5'] # '201963057861'
    folder = client.folder(streamstats_folder_id)
        #reach = 'NWIS_BoxSDK_test_02364500.csv'
    results = client.search().query(query = '.csv', limit=200, file_extensions=['csv'], ancestor_folders=[folder])
    for result in results:
        name = result.name[:-4]
        r_dict[name] = result.id
        print(f'The item ID is {result.id} and the item name is {name}')

NWIS_sites_al.h5


The item ID is 1182375176391 and the item name is NWIS_02469761
The item ID is 1182374635943 and the item name is NWIS_02479945
The item ID is 1182374151370 and the item name is NWIS_02466030
The item ID is 1182374222854 and the item name is NWIS_02465493
The item ID is 1182374400199 and the item name is NWIS_02455185
The item ID is 1182374318415 and the item name is NWIS_02454055
The item ID is 1182371653121 and the item name is NWIS_02453500
The item ID is 1182374241909 and the item name is NWIS_02453000
The item ID is 1182374026579 and the item name is NWIS_02438000
The item ID is 1182374960366 and the item name is NWIS_02423500
The item ID is 1182375055663 and the item name is NWIS_02425000
The item ID is 1182374624183 and the item name is NWIS_02423425
The item ID is 1182374297872 and the item name is NWIS_02418760
The item ID is 1182370687925 and the item name is NWIS_02413300
The item ID is 1182375149039 and the item name is NWIS_03592500
The item ID is 1182365328018 and the ite

In [93]:
#streamstats_folder_id =  'Streamflow_App'#Folder_Dictionary['NWIS_sites_ar.h5'] # '201963057861'
#folder = client.folder(streamstats_folder_id)
    #reach = 'NWIS_BoxSDK_test_02364500.csv'
results = client.search().query(query = 'Streamstats', limit=100, file_extensions=['csv'])#, ancestor_folders=[folder])
for result in results:
    name = result.name
    #r_dict[name] = result.id
    print(f'The item ID is {result.id} and the item name is {name}')

The item ID is 1056174272228 and the item name is StreamStats_sc.csv
The item ID is 1056175532301 and the item name is StreamStats_il.csv
The item ID is 1056173858702 and the item name is StreamStats_al.csv
The item ID is 1056173693016 and the item name is StreamStats_ne.csv
The item ID is 1056176784633 and the item name is StreamStats_pa.csv
The item ID is 1056176223313 and the item name is StreamStats_wi.csv
The item ID is 1056174471062 and the item name is StreamStats_in.csv
The item ID is 1056173143734 and the item name is StreamStats_md.csv
The item ID is 1056184853848 and the item name is StreamStats_va.csv
The item ID is 1056146119429 and the item name is StreamStats_ca.csv
The item ID is 1056147615371 and the item name is StreamStats_ny.csv
The item ID is 1056142696872 and the item name is StreamStats_la.csv
The item ID is 1056145384846 and the item name is StreamStats_wi.csv
The item ID is 1056151144857 and the item name is StreamStats_nj.csv
The item ID is 1056150754227 and t

# Code for extracting data from boxsdk api


In [6]:
NWM_Dictionary = pickle.load(open(cwd+'/Streamflow_App/NWM_site_keys.pkl', 'rb'))
NWM_Dictionary

{'NWM_789206': '1183265937780',
 'NWM_3296804': '1183231862880',
 'NWM_22274612': '1183266127472',
 'NWM_22206103': '1183266073186',
 'NWM_22203349': '1183264989758',
 'NWM_22038591': '1183266504340',
 'NWM_2189757': '1183272017729',
 'NWM_21712732': '1183265578147',
 'NWM_21661814': '1183264957097',
 'NWM_21655740': '1183265252320',
 'NWM_19621426': '1183334654887',
 'NWM_18634030': '1183265249673',
 'NWM_933140017': '1183265926480',
 'NWM_18566179': '1183266843936',
 'NWM_18566317': '1183264467049',
 'NWM_18523647': '1183264810741',
 'NWM_18229157': '1183265673998',
 'NWM_18215736': '1183273851420',
 'NWM_166759201': '1183263900199',
 'NWM_15823141': '1183264865131',
 'NWM_15810691': '1183231598622',
 'NWM_18225427': '1183231181063',
 'NWM_10854075': '1183264377621',
 'NWM_18094191': '1183253534832',
 'NWM_18094345': '1183270236060',
 'NWM_446386': '1183274352134',
 'NWM_2413994': '1183264927486',
 'NWM_22274808': '1183272331851',
 'NWM_22271288': '1183266031758',
 'NWM_22204871': '1

### Original code reading from directory

In [78]:
cwd

'C:\\Users\\rjohnson18\\Box\\CIROH\\Evaluation\\LULC_Streamflow'

In [15]:
state = 'ut'
NWIS_sites = pd.read_hdf(cwd+'/Data/StreamStats/StreamStats.h5', state)
NWIS_sites.columns

Index(['NWIS_site_id', 'NWIS_sitename', 'Drainage_area_mi2',
       'Mean_Basin_Elev_ft', 'Perc_Forest', 'Perc_Develop', 'Perc_Imperv',
       'Perc_Herbace', 'Perc_Slop_30', 'Mean_Ann_Precip_in', 'Ann_low_cfs',
       'Ann_mean_cfs', 'Ann_hi_cfs'],
      dtype='object')

In [20]:
U_state = state.upper()
Streamstats[Streamstats['state_id'] == U_state]

,NWIS_sitename,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,Ann_low_cfs,...,dec_long_va,alt_va,alt_acy_va,huc_cd,parm_cd,begin_date,end_date,drain_sqkm,state,state_id
NWIS_site_id,,,,,,,,,,,,,,,,,,,,,
09413500,"VIRGIN RIVER NEAR ST. GEORGE, UT",3980.000,5350.0,23.2,1.75,0.5300,11.40,1.48,14.7,62.60,...,-113.680516,2401.58,0.20,15010010.0,60.0,1950-10-01,2022-11-27,10303.00000,Utah,UT
09413900,"BEAVER DAM WASH NEAR ENTERPRISE, UT",59.000,5950.0,62.6,0.00,0.0530,1.88,35.00,19.0,2.05,...,-114.046646,4740.53,0.24,15010010.0,60.0,1991-10-01,2022-11-26,150.34950,Utah,UT
10163000,"PROVO RIVER AT PROVO, UT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.60,...,-111.711194,4503.00,1.60,NaN,60.0,1903-10-01,2022-11-26,1767.91200,Utah,UT
09180000,"DOLORES RIVER NEAR CISCO, UT",4570.000,7690.0,56.3,0.70,0.2300,8.55,6.57,20.4,101.80,...,-109.195114,4165.00,20.00,14030004.0,10.0,2006-07-18,2022-11-28,11837.85000,Utah,UT
09180500,"COLORADO RIVER NEAR CISCO, UT",0.001,4170.0,0.0,0.00,0.0000,0.00,13.70,NaN,2293.00,...,-109.293449,4092.35,0.24,14030005.0,10.0,2006-08-25,2022-11-28,NaN,Utah,UT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10215900,"MANTI CREEK BLW DUGWAY CREEK, NR MANTI, UT",25.700,9170.0,56.6,1.03,0.0616,14.50,34.80,23.8,10.60,...,-111.579916,6505.44,0.21,16030004.0,60.0,1964-10-01,2022-11-26,68.55569,Utah,UT
10219000,"SEVIER RIVER NEAR JUAB, UT",5160.000,7520.0,42.7,2.80,0.4500,7.97,1.01,17.2,119.60,...,-112.039656,4930.86,0.23,16030005.0,60.0,1911-10-01,2022-11-26,13101.59000,Utah,UT
10224000,"SEVIER RIVER NEAR LYNNDYL, UT",6010.000,7290.0,40.2,2.68,0.4700,8.32,4.26,17.0,135.00,...,-112.393834,4660.00,20.00,16030005.0,10.0,1980-10-01,1981-10-09,14417.10000,Utah,UT


In [21]:
Streamstats

,NWIS_sitename,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,Ann_low_cfs,...,dec_long_va,alt_va,alt_acy_va,huc_cd,parm_cd,begin_date,end_date,drain_sqkm,state,state_id
NWIS_site_id,,,,,,,,,,,,,,,,,,,,,
02378780,"D'OLIVE CREEK NEAR BRIDGEHEAD, AL",5.210000,NaN,NaN,53.90,11.50,NaN,NaN,NaN,9.99,...,-87.889647,NaN,NaN,NaN,60.0,2017-01-04,2020-10-25,NaN,Alabama,AL
02339495,OSELIGEE CREEK NEAR LANETT AL,86.300000,NaN,NaN,4.49,0.64,NaN,NaN,NaN,35.80,...,-85.196331,NaN,NaN,3130002.0,60.0,2006-07-26,2022-11-27,222.8292,Alabama,AL
02342500,"UCHEE CREEK NEAR FORT MITCHELL, AL.",322.000000,NaN,NaN,8.68,1.42,NaN,NaN,NaN,124.90,...,-85.014931,201.76,0.01,3130003.0,60.0,1946-10-01,2022-11-27,830.5740,Alabama,AL
02369800,BLACKWATER RIVER NEAR BRADLEY AL,0.044400,NaN,NaN,10.20,0.78,NaN,NaN,NaN,48.00,...,-86.709957,121.87,0.10,3140104.0,60.0,1967-10-01,2022-11-27,227.5129,Alabama,AL
02377570,"STYX RIVER NEAR ELSANOR, AL.",191.000000,NaN,NaN,4.72,0.74,NaN,NaN,NaN,157.90,...,-87.547206,NaN,NaN,3140106.0,60.0,1987-10-01,2022-11-27,495.4464,Alabama,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03213500,"PANTHER CREEK NEAR PANTHER, WV",31.000000,1780.0,NaN,3.50,NaN,NaN,NaN,NaN,17.20,...,-81.871111,1049.00,20.00,5070201.0,10.0,1973-05-10,1975-10-21,80.3709,West Virginia,WV
03214500,"TUG FORK AT KERMIT, WV",0.000077,626.0,NaN,100.00,NaN,NaN,NaN,NaN,534.60,...,-82.408758,574.07,20.00,5070201.0,10.0,2011-06-09,2018-10-19,3309.0550,West Virginia,WV
03206600,"EAST FORK TWELVEPOLE CREEK NEAR DUNLOW, WV",37.900000,1114.0,NaN,2.60,NaN,NaN,NaN,NaN,19.40,...,-82.305972,717.50,0.11,5090102.0,10.0,1974-03-14,1976-03-11,99.5850,West Virginia,WV


### Modified code to access data using BoxSDK

In [11]:
#Get Streamstats
streamstats_id = '1182325029983'
boxfile = bf(CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, ACCESS_TOKEN=ACCESS_TOKEN)
csvFile = boxfile.GetFile(streamstats_id).decode("utf-8")
ioFile = io.StringIO(csvFile)


Streamstats = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
Streamstats.pop('Unnamed: 0')
#NWIS site id's need to be 8 digits, < 8 need a zero added before.
#Make all NWIS sites correct 8 digit code
for i in np.arange(0, len(Streamstats),1):
            Streamstats.NWIS_site_id.loc[i] = str(Streamstats.NWIS_site_id.loc[i])
            if len(Streamstats.NWIS_site_id.loc[i]) < 8:
                Streamstats.NWIS_site_id.loc[i] = '0' + str(Streamstats.NWIS_site_id.loc[i])
            else:
                Streamstats.NWIS_site_id.loc[i] = str(Streamstats.NWIS_site_id.loc[i])

Streamstats.drop_duplicates(subset = 'NWIS_site_id', inplace = True)
Streamstats.set_index('NWIS_site_id', drop = True, inplace = True)

Streamstats

C:\Users\rjohnson18\AppData\Local\Temp\ipykernel_20776\340914653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Streamstats.NWIS_site_id.loc[i] = str(Streamstats.NWIS_site_id.loc[i])


,NWIS_sitename,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,Mean_Ann_Precip_in,Ann_low_cfs,...,dec_long_va,alt_va,alt_acy_va,huc_cd,parm_cd,begin_date,end_date,drain_sqkm,state,state_id
NWIS_site_id,,,,,,,,,,,,,,,,,,,,,
02378780,"D'OLIVE CREEK NEAR BRIDGEHEAD, AL",5.210000,NaN,NaN,53.90,11.50,NaN,NaN,NaN,9.99,...,-87.889647,NaN,NaN,NaN,60.0,2017-01-04,2020-10-25,NaN,Alabama,AL
02339495,OSELIGEE CREEK NEAR LANETT AL,86.300000,NaN,NaN,4.49,0.64,NaN,NaN,NaN,35.80,...,-85.196331,NaN,NaN,3130002.0,60.0,2006-07-26,2022-11-27,222.8292,Alabama,AL
02342500,"UCHEE CREEK NEAR FORT MITCHELL, AL.",322.000000,NaN,NaN,8.68,1.42,NaN,NaN,NaN,124.90,...,-85.014931,201.76,0.01,3130003.0,60.0,1946-10-01,2022-11-27,830.5740,Alabama,AL
02369800,BLACKWATER RIVER NEAR BRADLEY AL,0.044400,NaN,NaN,10.20,0.78,NaN,NaN,NaN,48.00,...,-86.709957,121.87,0.10,3140104.0,60.0,1967-10-01,2022-11-27,227.5129,Alabama,AL
02377570,"STYX RIVER NEAR ELSANOR, AL.",191.000000,NaN,NaN,4.72,0.74,NaN,NaN,NaN,157.90,...,-87.547206,NaN,NaN,3140106.0,60.0,1987-10-01,2022-11-27,495.4464,Alabama,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
03213500,"PANTHER CREEK NEAR PANTHER, WV",31.000000,1780.0,NaN,3.50,NaN,NaN,NaN,NaN,17.20,...,-81.871111,1049.00,20.00,5070201.0,10.0,1973-05-10,1975-10-21,80.3709,West Virginia,WV
03214500,"TUG FORK AT KERMIT, WV",0.000077,626.0,NaN,100.00,NaN,NaN,NaN,NaN,534.60,...,-82.408758,574.07,20.00,5070201.0,10.0,2011-06-09,2018-10-19,3309.0550,West Virginia,WV
03206600,"EAST FORK TWELVEPOLE CREEK NEAR DUNLOW, WV",37.900000,1114.0,NaN,2.60,NaN,NaN,NaN,NaN,19.40,...,-82.305972,717.50,0.11,5090102.0,10.0,1974-03-14,1976-03-11,99.5850,West Virginia,WV


In [41]:
#streamstats_folder_id =  '201992282490'  #Folder_Dictionary['NHD_segments_il.h5'] # 
#folder = client.folder(streamstats_folder_id)

#reach = 'NWIS_BoxSDK_test_02364500.csv'

#results = client.search().query(query=reach, limit=100, file_extensions=['csv'], ancestor_folders=[folder])

#r_dict = {}

#for result in results:
 #   r_dict[result.name] = result.id
    #print(f'The item ID is {result.id} and the item name is {result.name}')
    
#load dictionary
NWM_Dictionary = pickle.load(open(cwd+'/Streamflow_App/NWM_site_keys.pkl', 'rb'))
    
reachname = 'NWM_789206'
reachid = '1184303261277'#NWM_Dictionary[reachname]
    
boxfile = bf(CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, ACCESS_TOKEN=ACCESS_TOKEN)
csvFile = boxfile.GetFile(reachid).decode("utf-8")
ioFile = io.StringIO(csvFile)


df = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
df.pop('Unnamed: 0')
#NWIS site id's need to be 8 digits, < 8 need a zero added before.
df

,NWIS,NWIS_id
0,NWIS_07032000,1.183190e+12
1,NWIS_07047855,1.183080e+12
2,NWIS_07047942,1.183080e+12
3,NWIS_07047950,1.183080e+12
4,NWIS_07048550,1.183070e+12
...,...,...
4527,NWIS_02408150,1.182370e+12
4528,NWIS_02374950,1.182370e+12
4529,NWIS_02372250,1.182370e+12
4530,NWIS_02362240,1.182370e+12


In [84]:
csvFile = boxfile.GetFile('1184296622892').decode("utf-8")
ioFile = io.StringIO(csvFile)


df = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
df.pop('Unnamed: 0')
#NWIS site id's need to be 8 digits, < 8 need a zero added before.
df

,NWM,NWM_id
0,NWM_789206,1183265937780
1,NWM_3296804,1183231862880
2,NWM_22274612,1183266127472
3,NWM_22206103,1183266073186
4,NWM_22203349,1183264989758
...,...,...
4643,NWM_8423282,1184099312031
4644,NWM_5895480,1184095169388
4645,NWM_4352070,1184096985857
4646,NWM_19414869,1184095505409


In [ ]:
#Need to put the segment id dictionaries into a csv
dataname = 'NWIS'
dataid = dataname+'_id'
Searchable_filename = dataname+'_site_keys.csv'

#load dictionary
Dictionary = pickle.load(open(cwd+'/Streamflow_App/'+dataname+'_site_keys.pkl', 'rb'))

columns = [dataname, dataid]
DF = pd.DataFrame(columns = columns)

pbar = ProgressBar()
for site in pbar(Dictionary.keys()):
    #remove .csv from any id's
    if '.csv' in site:
        sitename = site[:-4]
    else:
        sitename = site
    siteinfo = [site, Dictionary[site]]
    #DF = DF.append(siteinfo, ignore_index = True)
    DF.loc[len(DF)] = siteinfo
    
#save as stringIO    
s_buf = StringIO()
DF.to_csv(s_buf)
new_file = client.folder('0').upload_stream(s_buf, Searchable_filename)
print(f'File "{new_file.name}" uploaded to Box with file ID {new_file.id}')

#updated_file = client.file('1184303261277').update_contents(s_buf)
#print(f'File "{updated_file.name}" with id: "{updated_file.id}" has been updated')
#File "NWM_site_keys.csv" uploaded to Box with file ID 1184296622892
#File "NWIS_site_keys.csv" uploaded to Box with file ID 1184303261277

In [ ]:
Streamstats = pd.read_hdf(cwd+'/Data/StreamStats/StreamStats3.h5', 'streamstats')

s_buf = StringIO()
Streamstats.to_csv(s_buf)
folder_id = subfolder.id # '201963057861'
new_file = client.folder(folder_id).upload_stream(s_buf, 'StreamStats.csv')
print(f'File "{new_file.name}" uploaded to Box with file ID {new_file.id}')

#Double check the folder uploaded.
streamstats_folder = subfolder.id # '201963057861'
items = client.folder(folder_id=streamstats_folder).get_items()
for item in items:
    print(f'{item.type.capitalize()} {item.id} is named "{item.name}"')

In [ ]:
#renaming files in a folder
path = cwd+'/Streamflow_App/Data/NWM_v2.1/'

for filename in os.listdir(path):
     if 'NHD' in filename:
        first = 'NWM_v2.1'
        last = filename[-12:]
        name = path+first+last
        print(name)
        os.rename(path+filename, name)

### Code for Getting StreamStats

In [ ]:
#Original file directory way
#Load streamstats wiht lat long to get geolocational information
Streamstats = pd.read_hdf(cwd+'/Data/StreamStats/StreamStats3.h5', 'streamstats')
Streamstats.drop_duplicates(subset = 'NWIS_site_id', inplace = True)
Streamstats.set_index('NWIS_site_id', drop = True, inplace = True)

#UPdated API way
streamstats_id = '1182325029983'
boxfile = bf(CLIENT_ID=CLIENT_ID, CLIENT_SECRET=CLIENT_SECRET, ACCESS_TOKEN=ACCESS_TOKEN)
csvFile = boxfile.GetFile(streamstats_id).decode("utf-8")
ioFile = io.StringIO(csvFile)


Streamstats = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
Streamstats.pop('Unnamed: 0')
#NWIS site id's need to be 8 digits, < 8 need a zero added before.
#Make all NWIS sites correct 8 digit code
for i in np.arange(0, len(Streamstats),1):
            Streamstats.NWIS_site_id.loc[i] = str(Streamstats.NWIS_site_id.loc[i])
            if len(Streamstats.NWIS_site_id.loc[i]) < 8:
                Streamstats.NWIS_site_id.loc[i] = '0' + str(Streamstats.NWIS_site_id.loc[i])
            else:
                Streamstats.NWIS_site_id.loc[i] = str(Streamstats.NWIS_site_id.loc[i])

Streamstats.drop_duplicates(subset = 'NWIS_site_id', inplace = True)
Streamstats.set_index('NWIS_site_id', drop = True, inplace = True)

Streamstats

In [ ]:
def Get_BoxSDK_data():
    print('Loading in ',State_Eval.model, ' predictions from ', State_Eval.startDT, ' to ', State_Eval.endDT)
    models = [State_Eval.model]
    model_dict ={}
    #Load in dataframe connecting the model nhd predictions
    #Get the Model prediction folder
    for model in models:
        results = client.search().query(query = model,type = 'folder', limit = 1)

        for result in results:
            name = result.name
            if name == model:
                #print(f'The item ID is {result.id} and the item name is {name}')
                model_dict[name] = result.id
                break

    #Get the Model's state prediction folder
    state_dict = {}
    folder_id =  model_dict[State_Eval.model]
    folder = client.folder(folder_id)
    state_key = State_Eval.model + '_state_keys.csv'
    results = client.search().query(query = state_key, limit=1, file_extensions=['csv'], ancestor_folders=[folder])

    for result in results:
        name = result.name
        if name == state_key:
            name = result.name[:-4]
            state_dict[name] = result.id
            #print(f'The item ID is {result.id} and the item name is {name}')
            break

    #Get state folder id
    state_file_id = state_dict[state_key[:-4]]
    boxfile = bf(CLIENT_ID=State_Eval.CLIENT_ID, 
                 CLIENT_SECRET=State_Eval.CLIENT_SECRET, 
                 ACCESS_TOKEN=State_Eval.ACCESS_TOKEN)
    csvFile = boxfile.GetFile(state_file_id).decode("utf-8")
    ioFile = io.StringIO(csvFile)

    state_file_list = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
    state_file_list.pop('Unnamed: 0')
    col_name = state_file_list.columns[0]
    col_id = state_file_list.columns[1]
    statefolder = 'NHD_segments_'+State_Eval.state+'.h5'
    state_file_list = state_file_list[state_file_list[col_name] == statefolder]

    reach_dict = {}

    folder = client.folder(str(state_file_list[col_id].values[0]))
    results = client.search().query(query = '.csv', limit=200, file_extensions=['csv'], ancestor_folders=[folder])
    for result in results:
        name = result.name[:-4]
        reach_dict[name] = result.id
        #print(f'The item ID is {result.id} and the item name is {name}')

    print('There are ', len(df), ' collocated NWIS/NHD reaches in ', State_Eval.category, ': ', State_Eval.size)

    #load NHD model data into dictionary,
    #I think we can make a function to multithread this to speed it up
    if len(reach_dict) >= 1:
        print('API connection successful, loading.')
        format = '%Y-%m-%d %H:%M:%S'
        pbar = ProgressBar()
        for reach in pbar(np.arange(0,len(df),1)):    
            NHD_reach = str(df.NHD_reachid[reach])
            reach_id = 'NWM_'+ NHD_reach
            csvFile = boxfile.GetFile(reach_dict[reach_id]).decode("utf-8")
            ioFile = io.StringIO(csvFile)
            Mod_flow = pd.read_csv(filepath_or_buffer = ioFile, delimiter = ",")
            Mod_flow['time'] ='12:00:00'
            Mod_flow['Datetime'] = pd.to_datetime(Mod_flow['Datetime']+ ' ' + Mod_flow['time'], format = format)
            Mod_flow.set_index('Datetime', inplace = True)
            Mod_flow = Mod_flow.loc[State_Eval.startDT:State_Eval.endDT]
            Mod_flow.pop('Unnamed: 0')
            cols = Mod_flow.columns
            State_Eval.Mod_data[NHD_reach] = Mod_flow[cols[0]]

    else:
        print('API connection was not successful, make sure the credentials are correct.')